Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

Izay misy hoe `YOUR CODE HERE` na "YOUR ANSWER HERE" ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References
Eto ilay references rehetra no apetraka

# https://www.kaggle.com/code/residentmario/soft-thresholding-with-lasso-regression/notebook
# https://xavierbourretsicotte.github.io/lasso_implementation.html

In [1]:
from random import randrange
import numpy as np
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.datasets import load_breast_cancer, load_diabetes
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.preprocessing import normalize


def grad_check_sparse(f, x, analytic_grad, num_checks=10, h=1e-5, error=1e-9):
    """
    sample a few random elements and only return numerical
    in this dimensions
    """

    for i in range(num_checks):
        ix = tuple([randrange(m) for m in x.shape])

        oldval = x[ix]
        x[ix] = oldval + h  # increment by h
        fxph = f(x)  # evaluate f(x + h)
        x[ix] = oldval - h  # increment by h
        fxmh = f(x)  # evaluate f(x - h)
        x[ix] = oldval  # reset

        grad_numerical = (fxph - fxmh) / (2 * h)
        grad_analytic = analytic_grad[ix]
        rel_error = abs(grad_numerical - grad_analytic) / (
            abs(grad_numerical) + abs(grad_analytic)
        )
        print(
            "numerical: %f analytic: %f, relative error: %e"
            % (grad_numerical, grad_analytic, rel_error)
        )
        assert rel_error < error

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [2]:
data = load_diabetes()
X, y = data.data, data.target

In [3]:
def mse_loss_vectorized(w, b, X, y):
    """
    MSE loss function WITHOUT FOR LOOPs , NO REGULARIZATION
    
    Returns a tuple of:
    - loss 
    - gradient with respect to weights w
    - gradient with respect to bias b
    """
    loss = 0.0
    dw = np.zeros_like(w)
    
    # YOUR CODE HERE
    y_pred = X.dot(w) + b
    
    # loss 
    loss = (1.0 / X.shape[0]) * ((y - y_pred) ** 2).sum()
    
    # gradient with respect to weights
    dw = - (2.0 / X.shape[0]) * X.T.dot(y - y_pred)
    
    # gradient with respect to bias
    db = - (2.0 / X.shape[0]) * ((y - y_pred)).sum()
        
    return loss, dw, np.array(db).reshape(1,)

In [4]:
def soft_threshold(x, delta):
    # YOUR CODE HERE
    if x > delta:
        return x - delta
    elif x < (- delta):
        return x + delta
    else:
        return 0

# Lasso Subgradient Descent

In [5]:
def l1_subgradient(w):
    """
    Subgradient of the L1 loss
    """
    dw = np.zeros_like(w)
    # YOUR CODE HERE
    dw = np.sign(w)
    return dw
    

def lasso_subgradient_mse_loss_vectorized(w, b, X, y, alpha):
    """
    MSE loss function adding the subgradient for w
    """
    loss, dw, db = mse_loss_vectorized(w, b, X, y)
    # Add the subgradient to dw
    # YOUR CODE HERE
    dw += alpha * l1_subgradient(w)
    return loss, dw, db

In [6]:
class LassolSubgradientDescent():
    def __init__(self,  alpha=0.1):
        self.w = None
        self.b = None
        self.alpha = alpha

    def train(self, X, y, learning_rate=1e-3, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
            choices = np.random.choice(N, batch_size, replace=False)
            
            X_batch = X[choices, :]
            y_batch = y[choices]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.  
            # YOUR CODE HERE
            self.w -= learning_rate * dw
            self.b -= learning_rate * db
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))
    
    
    def predict(self, X):
        # YOUR CODE HERE
        y_pred = X.dot(self.w) + self.b
        return y_pred
        
    def loss(self, X_batch, y_batch):
        return lasso_subgradient_mse_loss_vectorized(self.w, self.b, X_batch, y_batch, self.alpha)

In [7]:
model = LassolSubgradientDescent(alpha=0.1)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse = mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 28079.511818
iteration 10000 / 200000: loss 3122.122676
iteration 20000 / 200000: loss 3341.384934
iteration 30000 / 200000: loss 3256.374490
iteration 40000 / 200000: loss 3186.214769
iteration 50000 / 200000: loss 3029.594546
iteration 60000 / 200000: loss 3014.983863
iteration 70000 / 200000: loss 3296.357668
iteration 80000 / 200000: loss 2852.318491
iteration 90000 / 200000: loss 2658.461736
iteration 100000 / 200000: loss 2869.821959
iteration 110000 / 200000: loss 3143.727550
iteration 120000 / 200000: loss 2837.094063
iteration 130000 / 200000: loss 2965.729453
iteration 140000 / 200000: loss 3006.978603
iteration 150000 / 200000: loss 3003.509894
iteration 160000 / 200000: loss 2788.598590
iteration 170000 / 200000: loss 2950.872155
iteration 180000 / 200000: loss 3115.911219
iteration 190000 / 200000: loss 2686.326729
MSE scikit-learn: 2912.527492603619
MSE Coordinate descent model : 2888.7736561738557


In [8]:
model = LassolSubgradientDescent(alpha=2)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse = mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 30538.783813
iteration 10000 / 200000: loss 4057.922724
iteration 20000 / 200000: loss 3961.010640
iteration 30000 / 200000: loss 3751.226429
iteration 40000 / 200000: loss 3964.958744
iteration 50000 / 200000: loss 3719.298162
iteration 60000 / 200000: loss 3752.850105
iteration 70000 / 200000: loss 3520.351616
iteration 80000 / 200000: loss 3609.868670
iteration 90000 / 200000: loss 3486.128004
iteration 100000 / 200000: loss 4240.151209
iteration 110000 / 200000: loss 3560.023693
iteration 120000 / 200000: loss 3783.886059
iteration 130000 / 200000: loss 3976.368304
iteration 140000 / 200000: loss 3572.314639
iteration 150000 / 200000: loss 4011.673369
iteration 160000 / 200000: loss 3680.816407
iteration 170000 / 200000: loss 3251.212015
iteration 180000 / 200000: loss 3710.123773
iteration 190000 / 200000: loss 3930.049429
MSE scikit-learn: 5650.290772564547
MSE Coordinate descent model : 3810.4839627301453


# Lasso Proximal Gradient Descent (iterative soft thresholding algorithm or ISTA)

In [9]:
class LassoProximalGradientDescent():
    def __init__(self,  alpha=0.1):
        self.w = None
        self.b = None
        self.alpha = alpha

    def train(self, X, y, learning_rate=1e-3, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w is None: # Initialization
            self.w = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
            choices = np.random.choice(N, batch_size, replace=False)
            
            X_batch = X[choices, :]
            y_batch = y[choices]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.  
            # PROJECT THE GRADIENT FOR w USING soft_threshold
            # YOUR CODE HERE
            for i in range(d):
                self.w[i] = soft_threshold(self.w[i] - learning_rate * dw[i], self.alpha * learning_rate)

            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

    def predict(self, X):
        # YOUR CODE HERE
        y_pred = X.dot(self.w) + self.b
        return y_pred


    def loss(self, X_batch, y_batch):
        return mse_loss_vectorized(self.w, self.b, X_batch, y_batch)

In [13]:
model = LassoProximalGradientDescent(alpha=0.1)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 28303.829308
iteration 10000 / 200000: loss 25121.938963
iteration 20000 / 200000: loss 25208.246510
iteration 30000 / 200000: loss 28119.308063
iteration 40000 / 200000: loss 26199.436010
iteration 50000 / 200000: loss 25027.036846
iteration 60000 / 200000: loss 25689.155830
iteration 70000 / 200000: loss 26142.117494
iteration 80000 / 200000: loss 25037.314321
iteration 90000 / 200000: loss 26581.107467
iteration 100000 / 200000: loss 24789.983179
iteration 110000 / 200000: loss 26883.466918
iteration 120000 / 200000: loss 26101.222019
iteration 130000 / 200000: loss 25759.210875
iteration 140000 / 200000: loss 26050.222921
iteration 150000 / 200000: loss 25082.949517
iteration 160000 / 200000: loss 26760.403039
iteration 170000 / 200000: loss 27480.324144
iteration 180000 / 200000: loss 25656.546536
iteration 190000 / 200000: loss 24801.715024
MSE scikit-learn: 2912.527492603619
MSE Coordinate descent model : 26033.2230135236


AssertionError: 

In [14]:
model = LassoProximalGradientDescent(alpha=2)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 31065.844852
iteration 10000 / 200000: loss 28263.413042
iteration 20000 / 200000: loss 24970.214063
iteration 30000 / 200000: loss 26195.830894
iteration 40000 / 200000: loss 26470.020557
iteration 50000 / 200000: loss 28455.296849
iteration 60000 / 200000: loss 26030.475868
iteration 70000 / 200000: loss 26395.213199
iteration 80000 / 200000: loss 26339.430150
iteration 90000 / 200000: loss 26609.328456
iteration 100000 / 200000: loss 25704.796907
iteration 110000 / 200000: loss 26563.886070
iteration 120000 / 200000: loss 27837.891950
iteration 130000 / 200000: loss 24823.387382
iteration 140000 / 200000: loss 27741.275542
iteration 150000 / 200000: loss 25468.608732
iteration 160000 / 200000: loss 26058.032842
iteration 170000 / 200000: loss 27505.722605
iteration 180000 / 200000: loss 25080.703934
iteration 190000 / 200000: loss 27256.150303
MSE scikit-learn: 5650.290772564547
MSE Coordinate descent model : 26951.47898377782


AssertionError: 

# Lasso Projected Gradient Descent

In [15]:
class LassoProjectedGradientDescent():
    def __init__(self,  alpha=0.1):
        self.w_p = None
        self.w_n = None
        self.b = None
        self.alpha = alpha

    def train(self, X, y, learning_rate=1e-3, num_iters=100, batch_size=200, verbose=False):
        N, d = X.shape
        
        if self.w_p is None: # Initialization
            self.w_p = 0.001 * np.random.randn(d)
            self.w_n = 0.001 * np.random.randn(d)
            self.b = 0.0

        # Run stochastic gradient descent to optimize w
        
        loss_history = []
        for it in range(num_iters):
            X_batch = None
            y_batch = None
                                                               
            # Sample batch_size elements in X_batch and y_batch
            # X_batch shape is  (batch_size, d) and y_batch shape is (batch_size,)                                                                                          
            # Hint: Use np.random.choice to generate indices
            # YOUR CODE HERE
            choices = np.random.choice(N, batch_size, replace=False)
            
            X_batch = X[choices, :]
            y_batch = y[choices]
            
            # evaluate loss and gradient
            loss, dw, db = self.loss(X_batch, y_batch)

            # perform parameter update                                                                
            # Update the weights w using the gradient and the learning rate.  
            # Project for w_p and w_n
            # YOUR CODE HERE
            w = self.w_p - self.w_n
            grad = - X_batch.T.dot(y_batch - (X_batch.dot(w)))
    
            grad_wn = -grad + self.alpha * l1_subgradient(w)
            grad_wp = grad + self.alpha * l1_subgradient(w)
            
            self.w_p -= learning_rate * grad_wp
            self.w_n -= learning_rate * grad_wn
            
            self.b -= learning_rate * db
            
            
            if verbose and it % 10000 == 0:
                print("iteration %d / %d: loss %f" % (it, num_iters, loss))

    def predict(self, X):
        # YOUR CODE HERE
        y_pred = X.dot(self.w_p - self.w_n) + self.b
        return y_pred

    def loss(self, X_batch, y_batch):
        # YOUR CODE HERE
        return mse_loss_vectorized((self.w_p-self.w_n), self.b, X_batch, y_batch)

In [16]:
model = LassoProjectedGradientDescent(alpha=0.1)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 31644.673352
iteration 10000 / 200000: loss 3158.049157
iteration 20000 / 200000: loss 3005.114620
iteration 30000 / 200000: loss 2731.196819
iteration 40000 / 200000: loss 2958.091757
iteration 50000 / 200000: loss 2771.784644
iteration 60000 / 200000: loss 2863.866436
iteration 70000 / 200000: loss 2689.568956
iteration 80000 / 200000: loss 2880.072004
iteration 90000 / 200000: loss 2662.905580
iteration 100000 / 200000: loss 2670.338545
iteration 110000 / 200000: loss 3397.263616
iteration 120000 / 200000: loss 2758.700074
iteration 130000 / 200000: loss 2565.545559
iteration 140000 / 200000: loss 2804.618970
iteration 150000 / 200000: loss 3047.074908
iteration 160000 / 200000: loss 2993.571979
iteration 170000 / 200000: loss 2896.623305
iteration 180000 / 200000: loss 3352.199130
iteration 190000 / 200000: loss 2855.049434
MSE scikit-learn: 2912.527492603619
MSE Coordinate descent model : 2861.3285262785125


In [17]:
model = LassoProjectedGradientDescent(alpha=2)
model.train(X, y, learning_rate=1e-2,verbose=True, num_iters=200_000)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=True)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

iteration 0 / 200000: loss 30385.473770
iteration 10000 / 200000: loss 2804.456420
iteration 20000 / 200000: loss 2878.172622
iteration 30000 / 200000: loss 2739.772481
iteration 40000 / 200000: loss 2894.393529
iteration 50000 / 200000: loss 3018.068512
iteration 60000 / 200000: loss 2924.631573
iteration 70000 / 200000: loss 3000.226108
iteration 80000 / 200000: loss 2801.622795
iteration 90000 / 200000: loss 2974.558523
iteration 100000 / 200000: loss 2651.998150
iteration 110000 / 200000: loss 3121.434183
iteration 120000 / 200000: loss 2457.852914
iteration 130000 / 200000: loss 2838.367025
iteration 140000 / 200000: loss 2764.479709
iteration 150000 / 200000: loss 2535.280948
iteration 160000 / 200000: loss 2979.872411
iteration 170000 / 200000: loss 2949.383039
iteration 180000 / 200000: loss 2919.857808
iteration 190000 / 200000: loss 2813.317903
MSE scikit-learn: 5650.290772564547
MSE Coordinate descent model : 2863.4221601643208


# Lasso Coordinate Descent (no intercept)

In [18]:
class LassoCoordinateDescent():
    def __init__(self, alpha=0.1):
        self.w = None
        self.alpha = alpha

    def train(self, X, y, num_iters=1000):
        N, d = X.shape
        
        # YOUR CODE HERE
        #normalizing X in case it was not done before
        X = X / (np.linalg.norm(X, axis = 0)) 
        
        # Initialization
        self.w = np.ones((d,1))
            
        soft = np.vectorize(soft_threshold)
        
        #Looping until max number of iterations
        for i in range(num_iters): 

            #Looping through each coordinate
            for j in range(d):

                #Vectorized implementation
                X_j = X[:,j].reshape(-1,1)
                y_pred = (X @ self.w).reshape(-1)
                rho = X_j.T @ (y - y_pred.squeeze() + (self.w[j] * X_j).squeeze())
                # no fit_intercept
                self.w[j] = soft_threshold(rho, self.alpha)
                
        self.w.flatten()

    def predict(self, X): 
        # YOUR CODE HERE
        y_pred = X.dot(self.w)
        return y_pred

In [19]:
model = LassoCoordinateDescent(alpha=0.1)
model.train(X, y)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=0.1, fit_intercept=False)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

MSE scikit-learn: 26057.124496145752
MSE Coordinate descent model : 26004.303657948534


In [20]:
model = LassoCoordinateDescent(alpha=2)
model.train(X, y)
pred = model.predict(X)
mse= mean_squared_error(pred, y)

sk_model = Lasso(alpha=2, fit_intercept=False)
sk_model.fit(X, y)
sk_pred = sk_model.predict(X)
sk_mse = mean_squared_error(sk_pred, y)

print("MSE scikit-learn:", sk_mse)
print("MSE Coordinate descent model :", mse)
assert mse - sk_mse < 50

MSE scikit-learn: 28794.887776106683
MSE Coordinate descent model : 26006.422489676806
